# Data Science from Scratch Tutorial
## Statistics and Linear Regression
### Gail Muldoon 2016

This notebook uses titanic passengers and cricket chirp datasets to cover Statistics (Chapter 5) and Linear Regression (Chapter 14) from the book Data Science From Scratch by Joel Grus. 

## Load library dependencies and document software versions


In [ ]:
#Import pandas
import pandas as pd
# Import numpy
import numpy as np
# Import matplotlib
import matplotlib.pyplot as plt
#Import statsmodels
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
# Import the linear regression class
from sklearn.linear_model import LinearRegression

#Import seaborn
import seaborn as sns 
sns.set_style("white")
#make figures appear as they're made
%matplotlib inline 

In [ ]:
# Always good practice to note the version info
import sys
import matplotlib as mpl
print("python " + sys.version)
print("")
print("pandas " + str(pd.__version__))
print("numpy " + np.__version__)
#print("matplotlib " + mpl.__version__)
print("seaborn " + sns.__version__)

## Load Titanic data

In [ ]:
titanic = pd.read_csv("data/titanic_train.csv")
print(titanic.head(5))

## Chapter 5: Statistics

A statistic is just a number that describes some aspect of data. There are many basic stats that might be of interest for a dataset:

1. Central Tendencies: where is the data centered?
    * mean
    * median
    * mode
    * quantiles

2. Disperson: spread of data
    * range (max - min)
    * interquartile range (75th percentile - 25th percentile)
    * standard deviation / variance
    * outliers
    
3. Correlation: how do variables relate to one another?
    * covariance
    * correlation

In [ ]:
#Descriptive statistics of dataset
print(titanic.describe())

#### Question 1: Why does Age have a lower "count" than the other variables?
#### Question 2: Why are variables like Sex not shown in .describe()?

### More central tendencies
Might also be interested in median (the center value), which is not sensitive to outliers like the mean is.

In [ ]:
# get median for all numeric variables
titanic.median()

Note: It doesn't always make sense to take mean/median of a variable just because it's numeric
i.e. median of PassengerId doesn't have meaning

Or the mode, which is the value(s) that appear most often for a variable, whether they be numeric or not.

In [ ]:
titanic.mode()

### More dispersion
Can also get things like highest or lowest N values for a variable, like fare:

In [ ]:
titanicSorted = sorted(titanic.Fare)

#10 lowest fares (i.e. the first 10 after sorting)
print  titanicSorted[:10]

#10 highest fares (i.e. the last 10 after sorting)
print titanicSorted[-10:]

Or calculate arbitrary quantiles (i.e. percentiles):

In [ ]:
# compute 10% quantify, value for which 10% of data lies below
print(titanic.quantile(0.1))
print('')
# or 83.4% quantile, just in case
print(titanic.quantile(0.834))

#### Question 3: Choose your own quantile to calculate and interpret what it means.

Variance is another measure of dispersion which is just the squared standard deviation. Keep in mind standard deviation has the same units as the variable being measured, while the variance has units of the square of the value being measured. 

In [ ]:
#here's the variance
print(titanic.var())

#computing from standard deviation to check
var=titanic.std()**2
print(var)

Measures of dispersion tend to be sensitive to outliers. One measure of dispersion that's less sensitive to outliers is the interquartile range, which is the range between the 25th and 75th percentiles. 

In [ ]:
#compute interquartile range
interquartile = titanic.quantile(0.75) - titanic.quantile(0.25)
print(interquartile)

#compute the regular range for comparison
#Note range is already function in python, so I call it Range
Range = max(titanic.Fare) - min(titanic.Fare)
print(Range)

### Covariance 
From the book: "Whereas variance measures how a single variable deviates from its mean, covariance measures how two variables vary in tandem from their means." 

If the covariance is positive and relatively large, it means if one variable is above its mean value, the other variable tends to be above its mean or vice versa (both below their respective means). A large negative covariance indicates if one variable is above its mean, the other variable tends to be below its mean. Covariance close to 0 indicates no relationship.

Covariance is calculated using the following equation:

\begin{equation}
cov(x,y) = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{n-1} 
\end{equation}

$x$ = predictor variable, $\bar{x}$ = mean of predictor variable, $y$ = outcome variable, $\bar{y}$ = mean of outcome variable, $n$ = sample size

#### Question 4: What is the covariance of a variable with itself?
#### Question 5: What are the units of covariance?

In [ ]:
titanic.cov()

Notes:
1. As before, it doesn't necessarily make sense to take the covariance of a variable just because it has numeric values, e.g. PassengerId.

2. Covariance units depend on the variables being compared and may not be that easily interpretable.

3. "Large" covariance is relative to the variable values themselves.

4. A variable's covariance with itself is just its variance (squared standard deviation), e.g. for Fare:

In [ ]:
# covariance of Fare with itself
print(titanic["Fare"].cov(titanic.Fare))

# and variance of Fare
print(titanic["Fare"].var())

### Correlation
Unlike covariance, correlation is unitless and always between -1 (perfectly anti-correlated) and 1 (perfectly correlated). Correlation is basically covariance scaled by the respective standard deviations of each variable, which acts to normalize the values and leads to no units. This can make it easier to interpret. 

The equation for correlation is:

\begin{equation}
cor(x,y) = \frac{cov(x,y)}{std(x)std(y)}
\end{equation}

cov(x,y) is the covariance of the predictor and outcome, as before. $std$ is the standard deviation. 

If correlation is 0, there is no measureable relationship between the variables. Correlation is sensitive to outliers.

#### Question 6: What is the correlation of a variable with itself?

In [ ]:
titanic.corr()

### Simpson's Paradox and Causation

Sometimes correlation between variables can be misleading because of the presence of a confounding variable. This means that the _cause_ of variations in the two variables might actually be a third variable. Computing correlation assumes that all other variables are equally affecting the two variables you're comparing, which usually isn't true unless you've got a randomized experiment.

For example, there is a 95.86% (0.9586) correlation between per capita mozzarella cheese consumption and civil engineering PhDs awarded between 2000 and 2009. Chances are good that eating more mozzarella will not itself lead to more civil engineering PhDs in the world. There must be a missing variable linking the two.

<img src=images/mozzarellaPhDs.png align="center" width="75%"></div>

Another classic example is the positive correlation between murder and icecream consumption. Does eating more icecream lead to murderous tendencies? Does murdering someone lead to an icecream craving? Actually, both are independently correlated to higher outside temperatures.

It's always good to consider whether a correlation can be explained logically.

See more examples:

http://www.tylervigen.com/spurious-correlations



## Chapter 14: Simple Linear Regression

Linear regression is a way of computing a model that describes the linear relationship between variables. Simple linear regression uses only one predictor variable while multiple linear regression can include more predictors (also known as features) at once.

### Basics

The basic idea is that you want to find the line that best describes the relationship between a predictor (e.g. household income) and outcome (e.g. home value). Using the best fit line you calculate, you can predict the outcome for values of the predictor variable that may not be in your dataset.

You may know the equation of a line as:

$$ y=mx+b $$

In linear regression this is expressed as:

$$ y = \beta x + \alpha + \epsilon $$

$\beta$ is the slope of the line, $\alpha$ is the y-intercept, and $\epsilon$ describes the noise of your data around the line. $\beta$ tells you how much the outcome value increases for every unit increase in the predictor. $\alpha$ tells you what the outcome would be even if the predictor is 0. $\epsilon$ should be small assuming your data is actually linear (e.g. if there was high linear correlation between variables).

If you know $\alpha$ and $\beta$, you know the line that best estimates your data. In other words, the line is a model of your data!



### Finding the Best Fit Line using Least Squares

Values for $\alpha$ and $\beta$ (which define the regression line) are found using a least squares approach, illustrated below. In effect, the slope and intercept of the line are wiggled around until the offset between the data and the line are minimized. 

The distance between the data points and the line is measured by the "sum of squared errors", literally adding up the area of the squares formed from the difference between the measured outcome (from data) and predicted outcome (from estimated line). Whichever line (i.e. whichever values of $\alpha$ and $\beta$) gives the smallest sum of squared errors is the "least squares estimate", also known as the line of best fit. It's the best way to describe your linear data.

<img src=images/leastsq.gif align="center" width="75%"></div>
Source: dynamicgeometry.com

#### Question 7: Why is the titanic dataset not useful for practicing linear regression?

### When you should use simple linear regression
The titanic dataset isn't a good example for doing simple linear regression. To use this method we want a predictor and outcome which both have continuous values (rather than categorical like survival) and are linearly related to each other. 

Instead of titanic, we'll use a dataset of county data from the U.S. census. It was downloaded from here on 22 June 2016:
https://www.openintro.org/stat/data/cc.php

In [ ]:
county = pd.read_csv("data/countydata.txt",delimiter='\t')
county.head()

#### Question 8: How would you check to see if this data needs to be cleaned before we work with it? (Hint: Look for missing values, unfortunate column names, etc.)

#### Question 9: How do you know this dataset is suitable for simple linear regression?

We could consider the relationship between a number of variables here. For simple linear regression, we'll consider only one predictor and one outcome. To decide which to use, it might be useful to look at the correlation between sets of variables.

In [ ]:
# Correlation between per capita income and median value of owner-occupied homes is strongly positively correlated
print(county["per_capita_income"].corr(county.median_val_owner_occupied))
# Correlation between per capita income and poverty level is strongly negatively correlated.
print(county["per_capita_income"].corr(county.poverty))

#### Question 10: Try finding other combinations of variables on which you would want to try simple linear regression.

We'll work with just the first example, where higher per capita income is correlated with higher median home values. Feel free to try this method with your own example as well.

First, we can visually take a look at the data:

In [ ]:
#plot the data
sns.regplot('per_capita_income', 'median_val_owner_occupied', 
           data=county, 
           fit_reg=False)

We can see the positive correlation in the plot. 

Spoiler alert: seaborn will calculate and plot the regression line without any effort by just turning on the fit_reg flag. 

In [ ]:
#plot with regression line
sns.regplot('per_capita_income', 'median_val_owner_occupied', 
           data=county, 
           fit_reg=True)

However, it won't show you how good the regression line is or tell you what $\alpha$ and $\beta$ are so we can use them to predict specific estimates. But calculating the equation is also easy using the statsmodels function ols (ordinary least squares).

In [ ]:
#calculate the linear model
#the regression model is expressed as outcome ~ predictor
lm=smf.ols(formula='median_val_owner_occupied ~ per_capita_income', data=county).fit()

#print the coefficient values, alpha and beta
print(lm.params)

#### Question 11: The y-intercept, alpha, is negative! What does it mean to have a negative home value? 

It's reasonable to assume _any_ house will have a value of at least \$0. And we see this in the data: there are no data points for home values less than \$0. So for negative home values, our line is an extrapolation and doesn't make sense. 

In [ ]:
# the minimum home value in the data
county["median_val_owner_occupied"].min()

Instead, consider the case of our lowest possible home value, \$0. We can find the minimum per capita income that corresponds to the minimum home value by plugging in \$0 for the home value:

$$ homeValue = \beta * income - \alpha $$

$$ 0 = 12.15 * income - 140,340 $$

$$ income = $11,550.69 $$

This means we estimate that homeowners in this census dataset make a minimum per capita income of \$11,550.69 to own even the cheapest house. Of course, this is only an estimate and won't agree with the data perfectly. (Though why is any house listed at $0 value? Perhaps this is mis-reported or missing data?)

In [ ]:
# actual per capita income corresponding to home value of $0 in the dataset
county.per_capita_income[(county.median_val_owner_occupied == 0)]

The predict function will go the other way and give us our estimated home value for any income:

In [ ]:
lm.predict({'per_capita_income':[11550]}) #should be close to 0

#### Question 12: Try finding median home value estimates for your own choice of per capita income


#### Question 13: Our model estimates that for per capita income of \$75,000, median home value is \$771,423. Why might you be skeptical of this estimate?


In [ ]:
#predicted home value for per capita income of $75,000
print(lm.predict({'per_capita_income':[75000]})) 

You can plug in any number you choose for income, but be wary of out-of-sample estimates. That is, our model has only been evaluated for the range of incomes covered by our data. The same model may not be appropriate for values of income outside this range.

In [ ]:
#range of income in the data
print(county["per_capita_income"].min())
print(county["per_capita_income"].max())

### Evaluating the goodness of your model fit
To tell how well our line describes the data, we look at the $R^2$ value. $R^2$ is known as the _coefficient of determination_. It measures how much of the variance in the outcome variable (i.e. home value) is described by the model. $R^2$ ranges from 0 to 1 with a good fit being close to 1. In essence, a high R$^2$ values means that the predictor variable includes enough information to do a good job predicting what's going on with the outcome variable.

In [ ]:
#print the R^2 value
print(lm.rsquared)

This $R^2$ value for our income-home value data is relatively low. It tells us only 55% of the variance in median home value is explained by per capita income. This could suggest additional variables need to be included to better predict median home values. This is a job for multiple linear regression which we'll discuss in a future session.

Note the ols calculation also includes a lot more information for more complex statistical analysis:

In [ ]:
# uncomment for a lot more info from OLS calculation
#print(lm.summary()) 

#uncomment for a complete list of all the quantities of interest you can reference individually
#dir(lm)

### Assumptions

One tricky thing about using statistics correctly is to acknowledge and respect their limitations. Every statistical approach will have associated assumptions. Simple linear regression has four:

1. Linearity: the data should be best described by a line (instead of a curve, etc.)
    * Check with residual plot, should get random distribution around _y = 0_    
2. Homoscedasticity: variance should be constant for range of predictor values
    * Check with same residual plot, spread of residuals shouldn't get larger as a function of predictor value
3. Normality: errors should be normally distributed.
    * Check with Q-Q plot, should get the identity line
4. No autocorrelation of errors: each observation should be independent of the others
    * Check with residual plot, there shouldn't be a pattern

In [ ]:
#check for linearity
sns.residplot(county.median_val_owner_occupied,lm.resid)

This looks fine for linearity. Nonlinear data would show residuals with some curvature around the 0 line like this:

<img src=images/Nonlinear-residual.png align="center" width="50%"></div>

Source: http://docs.statwing.com/

In [ ]:
# check for constant variance (homoscedasticity)
sns.residplot(county.median_val_owner_occupied,lm.resid)

The spread about the 0 residual line doesn't seem to change all that much with home value, so the data appears homoscedastic. Heteroscedastic data would look something like this:


<img src=images/heteroskedastic.png align="center" width="50%"></div>

Source: http://www.econometricsbysimulation.com/

In [ ]:
#check for normality
fig = plt.figure()
fig=smg.qqplot(lm.resid,fit=True, line='s')

This shape of QQ plot is not that uncommon. The data is normal in the center of the distribution but our dataset has values more exteme than our model predicts. Because most of the data falls on the line, this isn't too poorly behaved, but note that the distribution isn't quite normal because of the thicker tails, particular at the higher end of the range.

In [ ]:
#check for error independence
#pd.tools.plotting.autocorrelation_plot(county)
#sns.regplot('per_capita_income',lm.resid,
#           data=county, 
#           fit_reg=False)

sns.residplot(county.median_val_owner_occupied,lm.resid)

We check for independence in the _errors_, not the outcome variable itself. This data looks to have uncorrelated errors, because they seem to vary randomly. 

When errors are correlated, each one will appear to follow from the last in a pattern.


<img src=images/correlatedErrors.png align="center" width="50%"></div>

Source: http://zoonek2.free.fr/UNIX/48_R/11.html